# TextGAN

In [ ]:
#Solo si se está usando Google Colab
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [ ]:
#Solo si se está usando Google Colab
#Solo ejecutar una vez, es para clonar el repo en drive (para tener las salidas generadas guardadas en el drive en caso de que se apague la maquina).
!git clone -b Thesis https://github.com/YuiZh0u/TextGAN-PyTorch.git

## Instalación de requisitos

In [ ]:
!cd TextGAN-PyTorch && pip install -r requirements.txt

In [ ]:
!pip install --upgrade gensim 

In [ ]:
!pip list

In [ ]:
!apt-get install libboost-all-dev

In [ ]:
!apt-get install build-essential cmake

In [ ]:
!git clone https://github.com/kpu/kenlm.git

In [ ]:
!cd kenlm && mkdir -p build && cd build && cmake .. && make -j 4

In [ ]:
import nltk
nltk.download('punkt')

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# import torch
torch.cuda.empty_cache()

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
torch.set_printoptions(profile="full")

## Logging

### Via `logging`

In [ ]:
from datetime import datetime
filename = datetime.now().strftime("%Y%m%d_%H%M")

In [ ]:
import logging
import sys

logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(message)s')
logger.setLevel(logging.INFO)

stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.INFO)
stdout_handler.setFormatter(formatter)

# fhandler = logging.FileHandler(filename='/home/jfrez/AI2/outputs_pynb/logs/log_{}.log'.format(filename), mode='w')
fhandler = logging.FileHandler(filename='/home/jfrez/AI2/outputs_pynb/logs/log_20220714_1508', mode='w')
fhandler.setFormatter(formatter)
fhandler.setLevel(logging.INFO)


logger.addHandler(fhandler)
logger.addHandler(stdout_handler)

### Via `%%tee`

In [5]:
from io import StringIO
import sys

from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.utils.capture import CapturedIO


class Tee(StringIO):
    def __init__(self, initial_value='', newline='\n', stream=None):
        self.stream = stream
        super().__init__(initial_value, newline)
    
    def write(self, data):
        if self.stream is not None:
            self.stream.write(data)
        
        super().write(data)


class capture_and_print_output(object):
    stdout = True
    stderr = True
    display = True
    
    def __init__(self, stdout=True, stderr=True, display=True):
        self.stdout = stdout
        self.stderr = stderr
        self.display = display
        self.shell = None
    
    def __enter__(self):
        from IPython.core.getipython import get_ipython
        from IPython.core.displaypub import CapturingDisplayPublisher
        from IPython.core.displayhook import CapturingDisplayHook
        
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr
        
        if self.display:
            self.shell = get_ipython()
            if self.shell is None:
                self.save_display_pub = None
                self.display = False
        
        stdout = stderr = outputs = None
        if self.stdout:
            stdout = sys.stdout = Tee(stream=sys.stdout)
        if self.stderr:
            stderr = sys.stderr = Tee(stream=sys.stderr)
        if self.display:
            self.save_display_pub = self.shell.display_pub
            self.shell.display_pub = CapturingDisplayPublisher()
            outputs = self.shell.display_pub.outputs
            self.save_display_hook = sys.displayhook
            sys.displayhook = CapturingDisplayHook(shell=self.shell,
                                                   outputs=outputs)
        
        return CapturedIO(stdout, stderr, outputs)
    
    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr
        if self.display and self.shell:
            self.shell.display_pub = self.save_display_pub
            sys.displayhook = self.save_display_hook


@magics_class
class CustomMagics(Magics):
    @magic_arguments.magic_arguments()
    @magic_arguments.argument('output', type=str, default='', nargs='?')
    @magic_arguments.argument('--no-stderr', action='store_true')
    @magic_arguments.argument('--no-stdout', action='store_true')
    @magic_arguments.argument('--no-display', action='store_true')
    @cell_magic
    def tee(self, line, cell):
        args = magic_arguments.parse_argstring(self.tee, line)
        out = not args.no_stdout
        err = not args.no_stderr
        disp = not args.no_display
        with capture_and_print_output(out, err, disp) as io:
            self.shell.run_cell(cell)
        if args.output:
            self.shell.user_ns[args.output] = io

In [6]:
from IPython import get_ipython

get_ipython().register_magics(CustomMagics)

## Run

In [ ]:
%%tee output
!cd TextGAN-PyTorch/run && python3 run_seqgan_fasttest.py 3 0

In [ ]:
from datetime import datetime
filename = datetime.now().strftime("%Y%m%d_%H%M")
with open('outputs_pynb/output{}.txt'.format(filename), 'w') as f:
    f.write(output.stdout)

## Extra

In [ ]:
!nvidia-smi

In [ ]:
!echo "hola"

In [ ]:
!cd TextGAN-PyTorch/utils && python3 visualization-custom.py

In [ ]:
!zip -r docencia_textgan.zip TextGAN-PyTorch